In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf


In [2]:
# Import cleaned VAERS dataset
vaers_df = pd.read_csv('VAERSDATA.csv', low_memory=False)
vaers_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'VAERSDATA.csv'

In [ ]:
# Check vaers_df column types
vaers_df.dtypes

In [ ]:
# change dates(objects)columns to datetime
vaers_df['RECVDATE'] = pd.to_datetime(vaers_df['RECVDATE'])
vaers_df['RPT_DATE'] = pd.to_datetime(vaers_df['RPT_DATE'])
vaers_df['DATEDIED'] = pd.to_datetime(vaers_df['DATEDIED'])
vaers_df['VAX_DATE'] = pd.to_datetime(vaers_df['VAX_DATE'])
vaers_df['ONSET_DATE'] = pd.to_datetime(vaers_df['ONSET_DATE'])
vaers_df['TODAYS_DATE'] = pd.to_datetime(vaers_df['TODAYS_DATE'])

vaers_df.dtypes

In [ ]:
# re-read categorical values# Categorical variable list
vaers_cat = vaers_df.dtypes[vaers_df.dtypes == 'object'].index.tolist()
vaers_cat

In [ ]:
# Unique values in each column
vaers_df[vaers_cat].nunique()

In [ ]:
# Print out "RECVDATE" value counts
recvdate_counts = vaers_df.RECVDATE.value_counts()
recvdate_counts